＊＊つい先日RNNの勉強を始めたので、そこまで難しいことをしていない方のノートブックをコピーして一部編集しています。
なるべく、知らないことがあったら説明やリンクを乗っけるようにしてます、

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import keras as keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers import Dense, Embedding, Bidirectional, Dropout, Flatten
from keras.optimizers import Adam, SGD
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
early_stop=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

**Dataset Load**

In [ ]:
train = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print('Training Dataset contain {} samples'.format(train.shape[0]))
print('Testing Dataset contain {} samples'.format(test.shape[0]))

We will drop 'id', 'keyword' and 'location columns as we are going to train an RNN only on text data.

In [ ]:
train = train.drop(['id', 'keyword', 'location'], axis=1)
test = test.drop(['id', 'keyword', 'location'], axis=1)

In [ ]:
y_train =  train['target'].values
X_train = train.drop(['target'], axis=1).values.reshape(len(train),)
X_test = test['text'].values.reshape(len(test),)

In [ ]:
print(X_test)

In [ ]:
print(y_train)
print(X_train)

**Preprocessing**

In [ ]:
total_tweets = np.concatenate((X_train, X_test))
print('Total tweets : ', len(total_tweets))

numpy.concatenate()の基本的な使い方

結合する配列ndarrayのリストを指定

結合する軸（次元）を指定: 引数axis

numpy.stack()で新たな軸（次元）に沿って結合

numpy.block()で配置を指定して結合

numpy.vstack()で縦に結合

numpy.hstack()で横に結合

numpy.dstack()で深さ方向に結合

In [ ]:
total_tweets

単語の文字列のindexを構築

In [ ]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(total_tweets)

# Vocbvulary Size
#　trian、testのツイートの総単語数（単語の次元数）を取得
vocab_size = len(tokenizer.word_index) + 1
print('Size of Vocabulary : ', vocab_size)

## [tokenizerの使いかたまとめ](https://weblabo.oscasierra.net/python/keras-tokenizer.html)

In [ ]:
# Maximum length for padding sequence
maxlen = max(len(x.split()) for x in total_tweets)
print('Maximum length of tweet : ', maxlen)

文字列を整数のインデックスのリストに変換

In [ ]:
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)

print('Text before tokenized')
print(X_train[0])
print('\nText after tokenized')
print(X_train_token[0])


要素の合わない配列に対して、0 で埋めるなどして配列のサイズを一致させる。
[参考記事](https://qiita.com/9ryuuuuu/items/2830fee559a41d00aa2b)


一番長いtweetの単語数に合わせる
paddding='post’
にすると後ろに0が追加される



In [ ]:
X_train_pad = pad_sequences(X_train_token, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_token, maxlen=maxlen, padding='post')

print('Tokenized text before padding')
print(X_train_token[0])
print('\nTokenized text after padding')
print(X_train_pad[0])

## モデルを作成していく
元ネタの方はBidirectional　LSTMで学習されていたのですが、いろいろと試してみたかったので下記の四つのモデルを作成して精度を比較しました

Bidirectionalって？？
普通のRNNだと前から順に次の文字を予想していくが、Bidirectionalをつけると前と後ろから読んで次に出てくる文字を予想できる。→精度がよくなるらしい

[参考記事](https://sleepless-se.net/2019/03/21/%E3%80%90python%E3%80%91bidirectionalrnn-create-sentence/)

In [ ]:

embed_units=100
hidden_units=128

model=Sequential()
model.add(Embedding(vocab_size, embed_units, input_length = maxlen))
model.add(SimpleRNN(hidden_units))
model.add(Dropout(0.2))
#model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid'))

model.summary()


In [ ]:
learning_rate = 0.0001

model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
batch_size_1= 150
batch_size_2= 300
batch_size_3= 700
num_itr = 5
es_cb=keras.callbacks.EarlyStopping( patience=0, verbose=0)
model_history = model.fit(X_train_pad, y_train, 
                          batch_size=batch_size_1, 
                          epochs=num_itr, 
                          validation_split=0.3,
                          callbacks=[es_cb])

## Bidirectional　SimpleRNN

In [ ]:


model_BRNN=Sequential()
model_BRNN.add(Embedding(vocab_size, embed_units, input_length = maxlen))
model_BRNN.add(Bidirectional(SimpleRNN(hidden_units)))
model_BRNN.add(Dropout(0.2))
#model.add(Flatten())
model_BRNN.add(Dense(256, activation='relu'))
model_BRNN.add(Dropout(0.2))
model_BRNN.add(Dense(1, activation='sigmoid'))

model_BRNN.summary()


In [ ]:
learning_rate = 0.0001

model_BRNN.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
model_BRNN_history = model_BRNN.fit(X_train_pad, y_train, 
                          batch_size=batch_size_3, 
                          epochs=num_itr, 
                          validation_split=0.2,
                          callbacks=[es_cb])

## LSTM

In [ ]:


model_LS = Sequential()
model_LS.add(Embedding(vocab_size, embed_units, input_length = maxlen))
model_LS.add(Bidirectional(LSTM(hidden_units)))
model_LS.add(Dropout(0.2))
#model.add(Flatten())
model_LS.add(Dense(256, activation='relu'))
model_LS.add(Dropout(0.2))
model_LS.add(Dense(1, activation='sigmoid'))

model_LS.summary()

In [ ]:
learning_rate = 0.0001

model_LS.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
model_LS_history = model_LS.fit(X_train_pad, y_train, 
                          batch_size=batch_size_3, 
                          epochs=num_itr, 
                          validation_split=0.4,
                          callbacks=[es_cb])

いろいろいじってみた結果
validation-splitを極端に上げるとたまにいい数字を出す、（よくわからん）
validation-splitを0.4ぐらいにして、バッチサイズをいじってみたがそんなに変わらない

# Bidirectional LSTM 

In [ ]:

model_BLS=Sequential()
model_BLS.add(Embedding(vocab_size, embed_units, input_length = maxlen))
model_BLS.add(Bidirectional(LSTM(hidden_units)))
model_BLS.add(Dropout(0.2))
#model.add(Flatten())
model_BLS.add(Dense(256, activation='relu'))
model_BLS.add(Dropout(0.2))
model_BLS.add(Dense(1, activation='sigmoid'))

model_BLS.summary()


In [ ]:
learning_rate = 0.0001

model_BLS.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:

model_BLS_history = model_LS.fit(X_train_pad, y_train, 
                          batch_size=batch_size_3, 
                          epochs=num_itr, 
                          validation_split=0.9,
                          callbacks=[es_cb])

いろいろいじってみた結果、
validation-splitを極端に挙げているのに、なかなか良い値を安定して出し続けている（なぜかはわからん）

In [ ]:
pred = model_LS.predict(X_test_pad)

In [ ]:
sub = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sub["target"] = pred
sub["target"] = sub["target"].apply(lambda x : 0 if x<=.5 else 1)

In [ ]:
sub

In [ ]:
sub.to_csv("submit_5.csv", index=False)

## 疑問点
validation-splitをを極端に上げるとval―accuracyが以上によくなる原因（所詮二値分類だから確率の問題？）
パラメータいじってもfitし直すごとにだいぶ値がかわるのでどのようにパラメータを調整すればいいのかわからない

## 分かったこと
自然言語処理の前処理の仕方
Bidirectional LSTM が（理由ははからないが）一番いいスコアを出すってこと
